<a href="https://colab.research.google.com/github/uzh-rpg/rpg_timelens/blob/daniel%2Ftest_code_release/TimeLens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TimeLens: Event-based Video Frame Interpolation

## Installation

First clone the repo and install an environment

In [ ]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go
!nvidia-smi

Wed Jun 16 13:00:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/gdrive
Google Drive connected.


In [2]:
!ls /content/gdrive

MyDrive  Shareddrives


# Install Dependencies and Repo

In [3]:
%cd /content
!git clone https://github.com/uzh-rpg/rpg_timelens
!wget http://rpg.ifi.uzh.ch/timelens/data/checkpoint.bin
!CUDA_VISIBLE_DEVICES=0

/content
Cloning into 'rpg_timelens'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 88 (delta 19), reused 79 (delta 17), pack-reused 0
Unpacking objects: 100% (88/88), done.
--2021-06-16 14:18:02--  http://rpg.ifi.uzh.ch/timelens/data/checkpoint.bin
Resolving rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)... 130.60.155.125
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.155.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 475379363 (453M) [application/octet-stream]
Saving to: ‘checkpoint.bin’

checkpoint.bin      100%[===================>] 453.36M   101MB/s    in 4.6s    

2021-06-16 14:18:07 (98.5 MB/s) - ‘checkpoint.bin’ saved [475379363/475379363]



In [4]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like TimeLens to do. If you do not have your own data, just leave the values below and it will download an example.

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "/content/rpg_timelens/out.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS =  960#@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/rpg_timelens/input_frames' #@param{type:"string"}
EVENTS_INPUT_DIR = '/content/rpg_timelens/input_events' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/rpg_timelens/output_frames' #@param{type:"string"}


# Set Up Example
If you do not have data to process, just leave the default values in the form above and the cell below will download an example.

In [5]:
# Example 
%%bash
wget http://rpg.ifi.uzh.ch/timelens/data/example_github.zip -O /content/example_github.zip
unzip /content/example_github.zip -d /content/
rm -rf /content/example_github.zip
mv /content/example/events /content/rpg_timelens/input_events
mv /content/example/images /content/rpg_timelens/input_frames
mkdir /content/rpg_timelens/output_frames

Archive:  /content/example_github.zip
   creating: /content/example/
   creating: /content/example/events/
  inflating: /content/example/events/000358.npz  
  inflating: /content/example/events/000371.npz  
  inflating: /content/example/events/000368.npz  
  inflating: /content/example/events/000376.npz  
  inflating: /content/example/events/000383.npz  
  inflating: /content/example/events/000351.npz  
  inflating: /content/example/events/000353.npz  
  inflating: /content/example/events/000382.npz  
  inflating: /content/example/events/000365.npz  
  inflating: /content/example/events/000391.npz  
  inflating: /content/example/events/000354.npz  
  inflating: /content/example/events/000352.npz  
  inflating: /content/example/events/000356.npz  
  inflating: /content/example/events/000398.npz  
  inflating: /content/example/events/000389.npz  
  inflating: /content/example/events/000367.npz  
  inflating: /content/example/events/000381.npz  
  inflating: /content/example/events/000360

--2021-06-16 14:18:17--  http://rpg.ifi.uzh.ch/timelens/data/example_github.zip
Resolving rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)... 130.60.155.125
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.155.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109862914 (105M) [application/zip]
Saving to: ‘/content/example_github.zip’

     0K .......... .......... .......... .......... ..........  0% 1.78M 59s
    50K .......... .......... .......... .......... ..........  0% 3.54M 44s
   100K .......... .......... .......... .......... ..........  0% 47.6M 30s
   150K .......... .......... .......... .......... ..........  0% 3.83M 29s
   200K .......... .......... .......... .......... ..........  0% 50.0M 24s
   250K .......... .......... .......... .......... ..........  0% 39.7M 20s
   300K .......... .......... .......... .......... ..........  0%  111M 18s
   350K .......... .......... .......... .......... ..........  0% 4.38M 18s
   400K .......... .......... ...

In [6]:
# find the fps of the input images 
import numpy as np
from os.path import join

timestamps = np.genfromtxt(join(FRAME_INPUT_DIR, "timestamp.txt"))
fps = int(1000000.0 / np.diff(timestamps).mean())
factor = int(TARGET_FPS / fps)
inserts = factor-1
print("Upsampling from ", fps, " to ", TARGET_FPS, " with ", inserts , " inserts.")



Upsampling from  159  to  960  with  5  inserts.


In [ ]:
%cd /content/rpg_timelens/
!mkdir $FRAME_OUTPUT_DIR
!python -m timelens.run_timelens ../checkpoint.bin $EVENTS_INPUT_DIR $FRAME_INPUT_DIR $FRAME_OUTPUT_DIR 0 $inserts

/content/rpg_timelens
mkdir: cannot create directory ‘/content/rpg_timelens/output_frames’: File exists
Processing .
100% 50/50 [00:01<00:00, 43.86it/s]
100% 50/50 [00:00<00:00, 150.52it/s]
Processing /content/rpg_timelens/output_frames/.
Counter: 0000
Events left:  6153 Events right:  31367
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
Events left:  12482 Events right:  25038
Events left:  18783 Events right:  18737
Events left:  25037 Events right:  12483
Events left:  31292 Events right:  6228
Counter: 0006
Events left:  6314 Events right:  30593
Events left:  12529 Events right:  24378
Events left:  18630 Events right:  18277
Events left:  24734 Events right:  12173
Events le